In [116]:
from simulator import Simulator, string_to_micro, micro_to_time

In [16]:
class AverageMidAlgo(object):   
    def __init__(self, session, date, tickers):
        # save the session information
        self.session = session
        self.date = date
        self.ticker = tickers[0]  # we only expect one ticker here so just save the one
        
        # set any global variables
        self.start_time = string_to_micro("9:30")
        self.end_time = string_to_micro("16:00")
        self.interval = string_to_micro("1s")
        self.sum_mid = 0.0
        self.num_mid = 0
        
        # subscribe to the ticker of interest, and set the first timer
        self.session.subscribe_ticker_all_feeds(self.ticker)
        self.session.add_timer(self.start_time, self.timer_callback)
        
    def timer_callback(self, time):
        # get the best bid and offer, compute the midmarket, and update averages
        bid, ask = self.session.get_inside_market(self.ticker)
        mid = (bid['price'] + ask['price']) / 2.0 / 1000000.0
        self.sum_mid += mid
        self.num_mid += 1
        ### HERE IS THE NEW LINE:
        #print "time:", micro_to_time(time), "midmarket:", mid, "running average:", self.sum_mid/self.num_mid 
        
        # reset the timer unless we are done
        if time < self.end_time:
            self.session.add_timer(time + self.interval, self.timer_callback) 
    
    def end(self):
        return self.sum_mid / self.num_mid

sim = Simulator(AverageMidAlgo)
sim.run("20170413", ["AAPL"])

Process Process-14:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/share/simulator/current/src/simulator.py", line 45, in runsim
    algo = self.Algo(session, date, tickers, **kwargs)
  File "<ipython-input-16-ef5e35c62f3b>", line 17, in __init__
    self.session.add_timer(self.start_time, self.start_callback)
AttributeError: 'AverageMidAlgo' object has no attribute 'start_callback'


In [120]:
import sys
from simulator import (
    Simulator, string_to_micro, micro_to_time,
    BUY, SELL, SHORT, EXCH_INET,
    BOOK_DEPTH1_PRICE, ORDER_EVENTS,
    )
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.ar_model import AR


class Peggy(object):
    def __init__(self, session, date, tickers, start_time, end_time, side1, side2, size):
        self.session = session
        self.date = date
        self.tickers = tickers
        self.ticker1 = self.tickers[0]
        self.ticker2 = self.tickers[1]
        self.start_time = start_time
        self.end_time = end_time
        self.interval = string_to_micro("1s")
        self.state = 'NULL'
        self.side1 = side1
        self.side2 = side2
        self.order_size = size
        self.buy_shares1 = 0
        self.buy_dollars = 0
        self.sell_shares1 = 0
        self.sell_dollars = 0
        self.buy_shares2 = 0
        self.sell_shares2 = 0
        
        
        self.dt = 1
        self.long_open = -1.25
        self.long_close = -0.50
        self.short_open = 1.25;
        self.short_close = 0.75
        self.training_size = 100.0
        
        self.session.add_timer(self.start_time, self.start_callback)
        
        self.results = {'time': []}
        for ticker in self.tickers:
            self.results[ticker] = []
            self.results['return {}'.format(ticker)] = []
        
        # subscribe to the ticker of interest, and set the first timer
        for ticker in self.tickers:
            self.session.subscribe_ticker_all_feeds(ticker)
    
    def start_callback(self, time):
        self.session.add_timer(time, self.timer_callback)
        for ticker in self.tickers:
            self.session.subscribe_event(ticker, BOOK_DEPTH1_PRICE + ORDER_EVENTS, self.event_callback)
        # reset the timer unless we are done 
        if time < self.end_time:
            self.session.add_timer(time + self.interval, self.start_callback) 
    
    def event_callback(self, ticker, event_params):
        self.process_executions(event_params)
        
    def timer_callback(self, time):
        # get the best bid and offer, compute the midmarket, and update averages
        #self.process_executions(event_params)
        
        self.results['time'].append(micro_to_time(time))
        bid1, ask1 = self.session.get_inside_market(self.ticker1)
        bid2, ask2 = self.session.get_inside_market(self.ticker2)
        
        self.results[self.ticker1].append(self.get_midmarket(self.ticker1) / 1000000.0)
        self.results[self.ticker2].append(self.get_midmarket(self.ticker2) / 1000000.0)
        
        if time > self.start_time + 10**6:
            self.results['return {}'.format(self.ticker1)].append(np.float(self.returns(self.results[self.ticker1][-2:])))
            self.results['return {}'.format(self.ticker2)].append(np.float(self.returns(self.results[self.ticker2][-2:])))
        
        if time > self.start_time + self.training_size * 10**6:   
            returns1 = self.results['return {}'.format(self.ticker1)][-100:]
            returns2 = self.results['return {}'.format(self.ticker2)][-100:]
            residuals, a,b = self.regress(returns1,returns2)
            
            kappa, m, sigma, sigmaeq = self.fitOU(residuals)
            
            s = self.sscore(m, sigmaeq)
            pos = self.buy_shares1 - self.sell_shares1 + self.buy_shares2 - self.sell_shares2
            self.order_size = 1
            
            if pos == 0:
                if s < -self.long_open:
                    self.side1 = BUY
                    self.side2 = SELL
                    price1 = ask1['price']
                    price2 = bid2['price']
                    self.session.add_order(self.ticker1, self.side1, self.order_size, price1, exchange=EXCH_INET)
                    self.session.add_order(self.ticker2, self.side2, self.order_size, price2, exchange=EXCH_INET)
                elif s > self.short_open:
                    self.side1 = SELL
                    self.side2 = BUY
                    price1 = bid1['price']
                    price2 = ask2['price']
                    self.session.add_order(self.ticker1, self.side1, self.order_size, price1, exchange=EXCH_INET)
                    self.session.add_order(self.ticker2, self.side2, self.order_size, price2, exchange=EXCH_INET)
            elif pos > 0 and s > -self.short_close:
                self.side1 = BUY
                self.side2 = SELL
                price1 = ask1['price']
                price2 = bid2['price']
                self.session.add_order(self.ticker1, self.side1, self.order_size, price1, exchange=EXCH_INET)
                self.session.add_order(self.ticker2, self.side2, self.order_size, price2, exchange=EXCH_INET)
            elif pos < 0 and s < self.long_close:
                self.side1 = SELL
                self.side2 = BUY
                price1 = bid1['price']
                price2 = ask2['price']
                self.session.add_order(self.ticker1, self.side1, self.order_size, price1, exchange=EXCH_INET)
                self.session.add_order(self.ticker2, self.side2, self.order_size, price2, exchange=EXCH_INET)
                
            print('s:' + str(s))
            print("Side: "+ str(self.side1))
            
    def process_executions(self, evp):
        if 'executed_orders' in evp:
            time = self.session.current_time()
            for ex in evp['executed_orders']:
                order = ex['order']
                side = order['side']
                ticker = order['ticker']
                if ticker == self.ticker1:
                    if side == 'B':
                        self.buy_shares1 += ex['quantity_executed']
                        self.buy_dollars += ex['quantity_executed'] * ex['price_executed']
                    else:
                        self.sell_shares1 += ex['quantity_executed']
                        self.sell_dollars += ex['quantity_executed'] * ex['price_executed']
                    pos = self.buy_shares1 - self.sell_shares1
                elif ticker == self.ticker2:
                    if side == 'B':
                        self.buy_shares2 += ex['quantity_executed']
                        self.buy_dollars += ex['quantity_executed'] * ex['price_executed']
                    else:
                        self.sell_shares2 += ex['quantity_executed']
                        self.sell_dollars += ex['quantity_executed'] * ex['price_executed']
                    pos = self.buy_shares2 - self.sell_shares2        
                pnl = self.get_pnl()
                print "{0} {1} {quantity_executed} {price_executed} {liquidity} {2} {3}".format(time, side, pos, pnl, **ex)
                

    def get_midmarket(self, ticker):
        bid, ask = self.session.get_inside_market(ticker)
        return (bid['price'] + ask['price']) / 2

    
    def get_pnl(self):
        # mark to mid
        mid1 = self.get_midmarket(self.ticker1)
        mid2 = self.get_midmarket(self.ticker2)
        pnl = self.sell_dollars - self.buy_dollars + (self.buy_shares1 - self.sell_shares1) * mid1 + (self.buy_shares2 - self.sell_shares2) * mid2
        return pnl
    
    def regress(self, returns1,returns2):
        x = np.asarray(returns1).reshape(-1,1)
        y = np.asarray(returns2).reshape(-1,1)
        model = LinearRegression()
        model.fit(x,y)
        a = model.intercept_[0]
        b = model.coef_[0,0]
        residuals = y-model.predict(x)
        return residuals, a,b
    
    def returns(self, midprices):
        return np.diff(midprices, axis=-1)/midprices[:-1]
    
    def fitOU(self, residual):
        ou = np.cumsum(residual)
        model = AR(ou)
        fittedmodel = model.fit(maxlag=1, disp=-1)  
        a = fittedmodel.params[0]
        b = fittedmodel.params[1]
        var =  fittedmodel.sigma2
        kappa = -np.log(b)/self.dt
        m = a/(1-np.exp(-kappa*self.dt))
        sigma = np.sqrt(var*2*kappa/(1-np.exp(-2*kappa*self.dt)))
        sigmaeq = np.sqrt(var/(1-np.exp(-2*kappa*self.dt)));
        return kappa, m, sigma, sigmaeq
    
    def sscore(self, m, sigmaeq):
        if sigmaeq != 0:
            return -m/sigmaeq
        elif m>0:
            return 10000000
        else:
            return -10000000
    
    def end(self):
        return self.get_pnl()

date = "20170413"
tickers = ['AAPL', 'MSFT']
start_time = string_to_micro("9:30")
end_time = string_to_micro("9:35")
side = BUY
size = 100
sim = Simulator(Peggy)
sim.run(date, tickers, use_om=True, start_time=start_time, end_time=end_time, side1=side, side2=side, size=size)


s:1.2321176075
Side: 66
s:1.14343055964
Side: 66
34302863047 B 1 141940000 A 1 -25000
34302863047 B 1 141940000 A 2 -50000
34302863047 B 1 141940000 A 3 -75000
34302863047 B 1 141940000 A 4 -100000
34302863047 B 1 141940000 A 5 -125000
34302863047 B 1 141940000 A 6 -150000
34302863047 B 1 141940000 A 7 -175000
34302863047 B 1 141940000 A 8 -200000
34302863047 B 1 141940000 A 9 -225000
34302863047 B 1 141940000 A 10 -250000
34302863047 B 1 141940000 A 11 -275000
34302863047 B 1 141940000 A 12 -300000
34302863047 B 1 141940000 A 13 -325000
34302863047 B 1 141940000 A 14 -350000
34302863047 B 1 141940000 A 15 -375000
34302863047 B 1 141940000 A 16 -400000
34302863047 B 1 141940000 A 17 -425000
34302863047 B 1 141940000 A 18 -450000
34302863047 B 1 141940000 A 19 -475000
34302863047 B 1 141940000 A 20 -500000
34302863047 B 1 141940000 A 21 -525000
34302863047 B 1 141940000 A 22 -550000
34302863047 B 1 141940000 A 23 -575000
34302863047 B 1 141940000 A 24 -600000
34302863047 B 1 141940000 A

34302863047 B 1 141940000 A 204 -5100000
34302863047 B 1 141940000 A 205 -5125000
34302863047 B 1 141940000 A 206 -5150000
s:1.42675705625
Side: 66
34303000248 B 1 141930000 R 207 -8270000
34303000248 B 1 141930000 R 208 -8300000
34303000248 B 1 141930000 R 209 -8330000
34303000248 B 1 141930000 R 210 -8360000
34303000248 B 1 141930000 R 211 -8390000
34303000248 B 1 141930000 R 212 -8420000
34303000248 B 1 141930000 R 213 -8450000
34303000248 B 1 141930000 R 214 -8480000
34303000248 B 1 141930000 R 215 -8510000
34303000248 B 1 141930000 R 216 -8540000
34303000248 B 1 141930000 R 217 -8570000
34303000248 B 1 141930000 R 218 -8600000
34303000248 B 1 141930000 R 219 -8630000
34303000248 B 1 141930000 R 220 -8660000
34303000248 B 1 141930000 R 221 -8690000
34303000248 B 1 141930000 R 222 -8720000
34303000248 B 1 141930000 R 223 -8750000
34303000248 B 1 141930000 R 224 -8780000
34303000248 B 1 141930000 R 225 -8810000
34303000248 B 1 141930000 R 226 -8840000
34303000248 B 1 141930000 R 227 

34303867249 S 1 65070000 A -93 -11825000
34303867249 S 1 65070000 A -94 -11830000
34303867249 S 1 65070000 A -95 -11835000
34303867249 S 1 65070000 A -96 -11840000
34303867249 S 1 65070000 A -97 -11845000
34303867249 S 1 65070000 A -98 -11850000
34303867249 S 1 65070000 A -99 -11855000
34303867249 S 1 65070000 A -100 -11860000
34303867249 S 1 65070000 A -101 -11865000
34303867249 S 1 65070000 A -102 -11870000
34303867249 S 1 65070000 A -103 -11875000
34303867249 S 1 65070000 A -104 -11880000
34303867249 S 1 65070000 A -105 -11885000
34303867249 S 1 65070000 A -106 -11890000
34303867249 S 1 65070000 A -107 -11895000
34303867249 S 1 65070000 A -108 -11900000
34303867249 S 1 65070000 A -109 -11905000
34303867249 S 1 65070000 A -110 -11910000
34303867249 S 1 65070000 A -111 -11915000
34303867249 S 1 65070000 A -112 -11920000
34303867249 S 1 65070000 A -113 -11925000
34303867249 S 1 65070000 A -114 -11930000
34303867249 S 1 65070000 A -115 -11935000
34303867249 S 1 65070000 A -116 -11940000

34303867249 S 1 65070000 A -289 -12805000
34303867249 S 1 65070000 A -290 -12810000
34303867249 S 1 65070000 A -291 -12815000
34303867249 S 1 65070000 A -292 -12820000
34303867249 S 1 65070000 A -293 -12825000
34303867249 S 1 65070000 A -294 -12830000
34303867249 S 1 65070000 A -295 -12835000
34303867249 S 1 65070000 A -296 -12840000
34303867249 S 1 65070000 A -297 -12845000
34303867249 S 1 65070000 A -298 -12850000
34303867249 S 1 65070000 A -299 -12855000
34303867249 S 1 65070000 A -300 -12860000
34303867249 S 1 65070000 A -301 -12865000
34303867249 S 1 65070000 A -302 -12870000
34303867249 S 1 65070000 A -303 -12875000
34303867249 S 1 65070000 A -304 -12880000
34303867249 S 1 65070000 A -305 -12885000
34303867249 S 1 65070000 A -306 -12890000
34303867249 S 1 65070000 A -307 -12895000
34303867249 S 1 65070000 A -308 -12900000
34303867249 S 1 65070000 A -309 -12905000
34303867249 S 1 65070000 A -310 -12910000
34303867249 S 1 65070000 A -311 -12915000
34303867249 S 1 65070000 A -312 -1

34393000248 B 1 65270000 R -196 -158030000
34393000248 B 1 65270000 R -195 -158035000
34393000248 B 1 65270000 R -194 -158040000
34393000248 B 1 65270000 R -193 -158045000
34393000248 B 1 65270000 R -192 -158050000
34393000248 B 1 65270000 R -191 -158055000
34393000248 B 1 65270000 R -190 -158060000
34393000248 B 1 65270000 R -189 -158065000
34393000248 B 1 65270000 R -188 -158070000
34393000248 B 1 65270000 R -187 -158075000
34393000248 B 1 65270000 R -186 -158080000
34393000248 B 1 65270000 R -185 -158085000
34393000248 B 1 65270000 R -184 -158090000
34393000248 B 1 65270000 R -183 -158095000
34393000248 B 1 65270000 R -182 -158100000
34393000248 B 1 65270000 R -181 -158105000
34393000248 B 1 65270000 R -180 -158110000
34393000248 B 1 65270000 R -179 -158115000
34393000248 B 1 65270000 R -178 -158120000
34393000248 B 1 65270000 R -177 -158125000
34393000248 B 1 65270000 R -176 -158130000
34393000248 B 1 65270000 R -175 -158135000
34393000248 B 1 65270000 R -174 -158140000
34393000248

...

34474000248 S 1 141670000 R -9976 -957935000
34474000248 S 1 141670000 R -9977 -957940000
34474000248 S 1 141670000 R -9978 -957945000
34474000248 S 1 141670000 R -9979 -957950000
34474000248 S 1 141670000 R -9980 -957955000
34474000248 S 1 141670000 R -9981 -957960000
34474000248 S 1 141670000 R -9982 -957965000
34474000248 S 1 141670000 R -9983 -957970000
34474000248 S 1 141670000 R -9984 -957975000
34474000248 S 1 141670000 R -9985 -957980000
34474000248 S 1 141670000 R -9986 -957985000
34474000248 S 1 141670000 R -9987 -957990000
34474000248 S 1 141670000 R -9988 -957995000
34474000248 S 1 141670000 R -9989 -958000000
34474000248 S 1 141670000 R -9990 -958005000
34474000248 S 1 141670000 R -9991 -958010000
34474000248 S 1 141670000 R -9992 -958015000
34474000248 S 1 141670000 R -9993 -958020000
34474000248 S 1 141670000 R -9994 -958025000
34474000248 S 1 141670000 R -9995 -958030000
34474000248 S 1 141670000 R -9996 -958035000
34474000248 S 1 141670000 R -9997 -958040000
3447400024

34476405047 S 1 65210000 A 9949 -958150000
34476405047 S 1 65210000 A 9948 -958145000
34476405047 S 1 65210000 A 9947 -958140000
34476405047 S 1 65210000 A 9946 -958135000
34476405047 S 1 65210000 A 9945 -958130000
34476405047 S 1 65210000 A 9944 -958125000
34476405047 S 1 65210000 A 9943 -958120000
34476405047 S 1 65210000 A 9942 -958115000
34476405047 S 1 65210000 A 9941 -958110000
34476405047 S 1 65210000 A 9940 -958105000
34476405047 S 1 65210000 A 9939 -958100000
34476405047 S 1 65210000 A 9938 -958095000
34476405047 S 1 65210000 A 9937 -958090000
34476405047 S 1 65210000 A 9936 -958085000
34476405047 S 1 65210000 A 9935 -958080000
34476405047 S 1 65210000 A 9934 -958075000
34476405047 S 1 65210000 A 9933 -958070000
34476405047 S 1 65210000 A 9932 -958065000
34476405047 S 1 65210000 A 9931 -958060000
34476405047 S 1 65210000 A 9930 -958055000
34476405047 S 1 65210000 A 9929 -958050000
34476405047 S 1 65210000 A 9928 -958045000
34476405047 S 1 65210000 A 9927 -958040000
34476405047

34487342247 B 1 141650000 A -10056 -258830000
34487342247 B 1 141650000 A -10055 -258835000
34487342247 B 1 141650000 A -10054 -258840000
34487342247 B 1 141650000 A -10053 -258845000
34487342247 B 1 141650000 A -10052 -258850000
34487342247 B 1 141650000 A -10051 -258855000
34487342247 B 1 141650000 A -10050 -258860000
34487342247 B 1 141650000 A -10049 -258865000
34487342247 B 1 141650000 A -10048 -258870000
34487342247 B 1 141650000 A -10047 -258875000
34487342247 B 1 141650000 A -10046 -258880000
34487342247 B 1 141650000 A -10045 -258885000
34487342247 B 1 141650000 A -10044 -258890000
34487342247 B 1 141650000 A -10043 -258895000
34487342247 B 1 141650000 A -10042 -258900000
34487342247 B 1 141650000 A -10041 -258905000
34487342247 B 1 141650000 A -10040 -258910000
34487342247 B 1 141650000 A -10039 -258915000
34487342247 B 1 141650000 A -10038 -258920000
34487342247 B 1 141650000 A -10037 -258925000
34487342247 B 1 141650000 A -10036 -258930000
34487342247 B 1 141650000 A -10035

34487342247 B 1 141650000 A -9872 -259750000
34487342247 B 1 141650000 A -9871 -259755000
34487342247 B 1 141650000 A -9870 -259760000
34487342247 B 1 141650000 A -9869 -259765000
34487342247 B 1 141650000 A -9868 -259770000
34487342247 B 1 141650000 A -9867 -259775000
34487342247 B 1 141650000 A -9866 -259780000
34487342247 B 1 141650000 A -9865 -259785000
34487342247 B 1 141650000 A -9864 -259790000
34487342247 B 1 141650000 A -9863 -259795000
34487342247 B 1 141650000 A -9862 -259800000
34487342247 B 1 141650000 A -9861 -259805000
34487342247 B 1 141650000 A -9860 -259810000
34487342247 B 1 141650000 A -9859 -259815000
34487342247 B 1 141650000 A -9858 -259820000
34487342247 B 1 141650000 A -9857 -259825000
34487342247 B 1 141650000 A -9856 -259830000
34487342247 B 1 141650000 A -9855 -259835000
34487342247 B 1 141650000 A -9854 -259840000
34487342247 B 1 141650000 A -9853 -259845000
34487342247 B 1 141650000 A -9852 -259850000
34487342247 B 1 141650000 A -9851 -259855000
3448734224

677501985000

In [122]:
print(micro_to_time(34487342247))

09:34:47.342247


In [112]:
import sys
from simulator import (
    Simulator, string_to_micro, micro_to_time,
    BUY, SELL, SHORT, EXCH_INET,
    BOOK_DEPTH1_PRICE, ORDER_EVENTS,
    )

class Peggy(object):
    def __init__(self, session, date, tickers, start_time, end_time, side, size):
        self.session = session
        self.date = date
        self.ticker = tickers[0]
        self.start_time = start_time
        self.end_time = end_time

        self.state = 'NULL'
        self.side = side
        self.order_size = size
        self.buy_shares = 0
        self.buy_dollars = 0
        self.sell_shares = 0
        self.sell_dollars = 0
        
        self.session.add_timer(self.start_time, self.start_callback)
        self.session.subscribe_ticker_all_feeds(self.ticker)
 
    def start_callback(self, time):
        self.session.subscribe_event(ticker, BOOK_DEPTH1_PRICE + ORDER_EVENTS, self.event_callback)

    def get_midmarket(self, ticker):
        bid, ask = self.session.get_inside_market(ticker)
        return (bid['price'] + ask['price']) / 2

    def get_pnl(self):
        # mark to mid
        mid = self.get_midmarket(self.ticker)
        pnl = self.sell_dollars - self.buy_dollars + (self.buy_shares - self.sell_shares) * mid
        return pnl

    def process_executions(self, evp):
        if 'executed_orders' in evp:
            time = self.session.current_time()
            for ex in evp['executed_orders']:
                order = ex['order']
                side = order['side']
                ticker = order['ticker']
                if side == 'B':
                    self.buy_shares += ex['quantity_executed']
                    self.buy_dollars += ex['quantity_executed'] * ex['price_executed']
                else:
                    self.sell_shares += ex['quantity_executed']
                    self.sell_dollars += ex['quantity_executed'] * ex['price_executed']
                pos = self.buy_shares - self.sell_shares
                pnl = self.get_pnl()
                print "{0} {1} {quantity_executed} {price_executed} {liquidity} {2} {3}".format(time, side, pos, pnl, **ex)
    
    
    def event_callback(self, ticker, event_params):
        self.process_executions(event_params)
        bid, ask = self.session.get_inside_market(ticker)
        pos = self.buy_shares - self.sell_shares
        if self.buy_shares == 0:
            self.side = BUY
            self.order_size = 100
        elif pos < 0:
            self.side = BUY
            self.order_size = 200
        else:
            self.side = SELL
            self.order_size = 200
        price = bid['price'] if side == BUY else ask['price']
        orders = self.session.get_orders_by_ticker(self.ticker)

        if not orders:
            self.session.add_order(self.ticker, self.side, self.order_size, price, exchange=EXCH_INET)
        else:
            assert len(orders) == 1
            order = orders[0]
            if order['price'] != price:
                self.session.cancel_order(order['order_id'])
    
    def end(self):
        return self.get_pnl()

date = "20150121"
ticker = "IVV"
start_time = string_to_micro("9:30")
end_time = string_to_micro("9:31")
side = BUY
size = 100
sim = Simulator(Peggy)
sim.run(date, [ticker], use_om=True, start_time=start_time, end_time=end_time, side=side, size=size)

IVV
34200062196 B 100 202920000 A 100 500000
IVV
34200062397 S 100 202910000 R 0 -1000000
IVV
34200154996 S 100 202880000 A -100 -1500000
IVV
34200155196 S 100 202880000 A -200 -6000000
IVV
34200213396 B 100 202920000 A -100 -9000000
IVV
34200227796 B 100 202920000 A 0 -9000000
IVV
34200286596 S 100 202910000 R -100 -9000000
IVV
34200400796 B 60 202930000 A -40 -11200000
IVV
34200401796 B 100 202930000 A 60 -10700000
IVV
34200440196 B 40 202930000 A 100 -11500000
IVV
34200463196 S 200 202900000 A -100 -15000000
IVV
34202848799 B 100 202920000 A 0 -16000000
IVV
34203097196 B 100 202920000 A 100 -16500000
IVV
34203097404 S 200 202910000 R -100 -17500000
IVV
34203101396 B 100 202910000 A 0 -17000000
IVV
34203101396 B 100 202910000 A 100 -17000000
IVV
34203221598 S 200 202870000 R -100 -23000000
IVV
34203407796 B 100 202900000 A 0 -24000000
IVV
34203434196 B 100 202900000 A 100 -23500000
IVV
34203519196 S 7 202900000 A 93 -24000000
IVV
34203628196 S 100 202900000 A -7 -24035000
IVV
3420362

IVV
34429593996 S 100 202520000 A -190 -373450000
IVV
34429596396 B 100 202530000 A -90 -374850000
IVV
34429598796 B 100 202530000 A 10 -374450000
IVV
34429598997 S 100 202520000 R -90 -374500000
IVV
34431026396 B 100 202500000 A 10 -372650000
IVV
34431027396 B 100 202500000 A 110 -374350000
IVV
34431027603 S 200 202470000 R -90 -377350000
IVV
34436456596 B 200 202470000 A 110 -376550000
IVV
34436456996 S 200 202460000 A -90 -377550000
IVV
34437847996 B 200 202470000 A 110 -378550000
IVV
34437853397 S 200 202430000 R -90 -383750000
IVV
34437897996 B 100 202460000 A 10 -385200000
IVV
34437897996 B 100 202460000 A 110 -386200000
IVV
34437898200 S 100 202440000 R 10 -387300000
IVV
34437898396 S 100 202440000 A -90 -388200000
IVV
34439522596 B 200 202440000 A 110 -388950000
IVV
34439525197 S 200 202400000 R -90 -393500000
IVV
34448388596 B 200 202510000 A 110 -402700000
IVV
34448422996 S 100 202470000 A 10 -405750000
IVV
34448422996 S 100 202470000 A -90 -408250000
IVV
34459302396 B 100 20

IVV
34583585996 B 200 203060000 A 10 -728200000
IVV
34583774597 S 200 203010000 R -190 -733350000
IVV
34586688196 B 200 203060000 A 10 -738200000
IVV
34586767797 S 100 203040000 A -90 -738300000
IVV
34590794196 B 100 203120000 A 10 -745400000
IVV
34590799196 B 100 203120000 A 110 -747150000
IVV
34590801796 S 200 203060000 R -90 -754800000
IVV
34591084596 B 100 203110000 A 10 -756550000
IVV
34591164596 B 100 203110000 A 110 -757700000
IVV
34591171197 S 100 203070000 R 10 -761000000
IVV
34591189796 S 200 203080000 A -190 -763750000
IVV
34599799396 B 200 203220000 A 10 -787750000
IVV
34599803598 S 200 203160000 R -190 -790000000
IVV
34600554196 B 200 203160000 A 10 -788200000
IVV
34600559396 S 200 203110000 R -190 -793350000
IVV
34601597796 B 200 203140000 A 10 -794500000
IVV
34601918396 S 200 203080000 R -190 -798700000
IVV
34603594596 B 200 203110000 A 10 -800750000
IVV
34603597396 S 200 203060000 R -190 -805850000
IVV
34605932796 B 100 203080000 A -90 -805350000
IVV
34605932996 B 100 2

IVV
34868395196 B 100 204170000 A 14 -1688270000
IVV
34868395397 S 200 204160000 R -186 -1689270000
IVV
34872218596 B 6 204190000 A -180 -1694820000
IVV
34872320196 B 194 204190000 A 14 -1694270000
IVV
34872807996 S 200 204110000 R -186 -1699690000
IVV
34886076796 B 100 204200000 A -86 -1711780000
IVV
34886773396 B 200 204210000 A 114 -1716060000
IVV
34887003396 S 200 204120000 A -86 -1726340000
IVV
34893132796 B 100 204140000 A 14 -1724620000
IVV
34893671396 B 100 204140000 A 114 -1726900000
IVV
34893683396 S 200 204080000 R -86 -1733610000
IVV
34896268196 B 200 204090000 A 114 -1734600000
IVV
34896271397 S 200 204030000 R -86 -1741310000
IVV
34898450996 B 200 204060000 A 114 -1743450000
IVV
34898466996 S 200 204010000 A -86 -1749160000
IVV
34904554196 B 200 204030000 A 114 -1751440000
IVV
34904555997 S 200 203980000 R -86 -1757010000
IVV
34905445796 B 200 204010000 A 114 -1757440000
IVV
34905451796 S 200 203980000 R -86 -1762150000
IVV
34906938596 B 200 204010000 A 114 -1765720000
IV

IVV
35032991396 S 100 203500000 A -186 -2147510000
IVV
35032998996 B 100 203500000 A -86 -2147440000
IVV
35033047396 B 100 203500000 A 14 -2146580000
IVV
35033048796 S 100 203490000 A -86 -2146720000
IVV
35033190796 B 100 203510000 A 14 -2148370000
IVV
35034070396 S 200 203520000 R -186 -2149230000
IVV
35037676996 B 100 203690000 A -86 -2180350000
IVV
35038128996 B 200 203700000 A 114 -2183060000
IVV
35038143996 S 200 203670000 A -86 -2184630000
IVV
35038759396 B 200 203670000 A 114 -2185340000
IVV
35039002796 S 200 203670000 A -86 -2185490000
IVV
35041779996 B 100 203710000 A 14 -2188060000
IVV
35041779996 B 100 203710000 A 114 -2191060000
IVV
35041794396 S 200 203670000 A -86 -2193060000
IVV
35042051598 B 100 203680000 A 14 -2193130000
IVV
35042051598 B 100 203680000 A 114 -2193630000
IVV
35042073996 S 200 203670000 A -86 -2195060000
IVV
35045677196 B 100 203680000 A 14 -2195060000
IVV
35045686196 B 100 203680000 A 114 -2197340000
IVV
35045706196 S 200 203630000 A -86 -2202480000
IVV

35207309996 S 200 203390000 A -86 -2715270000
IVV
35212872796 B 200 203500000 A 114 -2724860000
IVV
35213086996 S 200 203470000 A -86 -2727290000
IVV
35213417196 B 200 203470000 A 114 -2727140000
IVV
35216837796 S 200 203420000 A -86 -2732560000
IVV
35222157796 B 200 203510000 A 114 -2742290000
IVV
35222167397 S 200 203470000 R -86 -2745290000
IVV
35225464596 B 200 203500000 A 114 -2748290000
IVV
35225585996 S 200 203470000 A -86 -2751290000
IVV
35227879196 B 200 203480000 A 114 -2753140000
IVV
35228122796 S 200 203440000 A -86 -2756280000
IVV
35230216396 B 200 203450000 A 114 -2757990000
IVV
35230260996 S 200 203420000 A -86 -2760560000
IVV
35232807796 B 200 203430000 A 114 -2762840000
IVV
35234063196 S 200 203370000 A -86 -2768690000
IVV
35235017596 B 200 203380000 A 114 -2770540000
IVV
35235021597 S 200 203330000 R -86 -2775680000
IVV
35235827596 B 200 203340000 A 114 -2775960000
IVV
35237014596 S 200 203290000 A -86 -2782240000
IVV
35238588596 B 200 203330000 A 114 -2786240000
IVV


IVV
35372844196 S 200 203250000 A -123 -3211830000
IVV
35374341396 B 200 203250000 A 77 -3211370000
IVV
35374347396 S 200 203230000 A -123 -3213370000
IVV
35379147596 B 200 203280000 A 77 -3219445000
IVV
35379321596 S 200 203250000 A -123 -3221830000
IVV
35379769196 B 200 203260000 A 77 -3223370000
IVV
35379876996 S 200 203220000 A -123 -3226755000
IVV
35380233396 B 200 203240000 A 77 -3228910000
IVV
35380237596 S 200 203200000 A -123 -3232295000
IVV
35381313396 B 200 203200000 A 77 -3231220000
IVV
35381318396 S 200 203160000 R -123 -3235990000
IVV
35383549796 B 200 203180000 A 77 -3236760000
IVV
35383553396 S 200 203150000 A -123 -3240145000
IVV
35385109396 B 200 203200000 A 77 -3245220000
IVV
35385501996 S 200 203160000 A -123 -3250605000
IVV
35387365796 B 200 203230000 A 77 -3257295000
IVV
35387440996 S 200 203190000 A -123 -3261680000
IVV
35390200596 B 200 203230000 A 77 -3265295000
IVV
35390273196 S 200 203180000 R -123 -3270450000
IVV
35392755396 B 200 203200000 A 77 -3271220000


35692543796 B 200 203480000 A 177 -3823545000
IVV
35692543997 S 100 203460000 R 77 -3824815000
IVV
35693310396 S 200 203400000 R -123 -3831510000
IVV
35695648396 B 100 203440000 A -23 -3834085000
IVV
35695648596 B 100 203440000 A 77 -3834355000
IVV
35695658396 S 200 203410000 A -123 -3839355000
IVV
35695833996 B 200 203430000 A 77 -3840280000
IVV
35695837196 S 200 203390000 R -123 -3843665000
IVV
35704282596 B 141 203580000 A 18 -3865100000
IVV
35709003796 S 200 203600000 A -182 -3866650000
IVV
35711844596 B 200 203650000 A 18 -3874110000
IVV
35711844798 S 100 203630000 R -82 -3874290000
IVV
35714003396 B 100 203650000 A 18 -3875930000
IVV
35714006196 B 100 203650000 A 118 -3877110000
IVV
35714016396 S 200 203600000 R -82 -3883470000
IVV
35726977996 B 200 203680000 A 118 -3890160000
IVV
35727003396 S 200 203640000 A -82 -3894340000
IVV
35729513396 B 200 203680000 A 118 -3898160000
IVV
35730901996 S 200 203640000 A -82 -3902750000
IVV
35737343196 B 200 203680000 A 118 -3906160000
IVV
35

Process Process-108:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/share/simulator/current/src/simulator.py", line 47, in runsim
    session.run()
  File "<ipython-input-112-0a7a31fa9a4b>", line 59, in event_callback
    def event_callback(self, ticker, event_params):
KeyboardInterrupt


KeyboardInterrupt: 